# 1.网页类型分析
## 1.1 网页类型统计
### 1.1.1 连接数据库

In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/wangye?charset=utf8')
sql = pd.read_sql('all_gzdata', engine, chunksize = 10000)
'''
用create_engine建立连接，连接地址的意思依次为“数据库格式（mysql）+程序名（pymysql）+账号密码@地址端口/数据库名（test）”，最后指定编码为utf8；
all_gzdata是表名，engine是连接数据的引擎，chunksize指定每次读取1万条记录。这时候sql是一个容器，未真正读取数据。
'''

D:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 478")
  result = self._query(query)


'\n用create_engine建立连接，连接地址的意思依次为“数据库格式（mysql）+程序名（pymysql）+账号密码@地址端口/数据库名（test）”，最后指定编码为utf8；\nall_gzdata是表名，engine是连接数据的引擎，chunksize指定每次读取1万条记录。这时候sql是一个容器，未真正读取数据。\n'

### 1.1.2 统计fullURLId的数量

In [2]:
counts = [ i['fullURLId'].value_counts() for i in sql] #按次10000存取，逐块统计

In [3]:
counts = pd.concat(counts).groupby(level=0).sum() #合并统计结果，把相同的统计项合并（即按index分组并求和）
counts = counts.reset_index() #重新设置index，将原来的index作为counts的一列。
counts.columns = ['index', 'num'] #重新设置列名，主要是第二列，默认为0
counts

,index,num
0,101001,633
1,101002,983
2,101003,49431
3,101004,19
4,101005,3
5,101006,14
6,101007,17
7,101008,33
8,101009,107
9,102001,215


### 1.1.3 提取前3个数字做网页类别

In [4]:
counts['type'] = counts['index'].str.extract('(\d{3})') #提取前三个数字作为类别id
counts

,index,num,type
0,101001,633,101
1,101002,983,101
2,101003,49431,101
3,101004,19,101
4,101005,3,101
5,101006,14,101
6,101007,17,101
7,101008,33,101
8,101009,107,101
9,102001,215,102


In [6]:
counts_ = counts[['type', 'num']].groupby('type').sum() #按类别合并
counts_.sort_values('num', ascending = False) #降序排列
counts_['percentage'] = (counts_['num']/counts_['num'].sum())*100
counts_.to_excel('./tmp/1_1_3type_counts.xlsx')
counts_

,num,percentage
type,,
101,51240,47.543493
102,2714,2.518209
103,313,0.290420
106,664,0.616098
107,24707,22.924611
199,26069,24.188355
301,2068,1.918812


## 1.2 各网页类型中小类别占比

In [7]:
# 每个大类别下面小类别占比
a = counts.set_index(['type'])
print(a)
b = counts.groupby('type').sum()
print(b)
c = pd.merge(a,b,left_index=True,right_index=True)
print(c)

        index    num
type                
101    101001    633
101    101002    983
101    101003  49431
101    101004     19
101    101005      3
101    101006     14
101    101007     17
101    101008     33
101    101009    107
102    102001    215
102    102002   2164
102    102003    152
102    102004     35
102    102005     42
102    102006     19
102    102007     31
102    102008     30
102    102009     26
103    103002     21
103    103003    292
106    106001    664
107    107001  24707
199   1999001  26069
301    301001   2068
        num
type       
101   51240
102    2714
103     313
106     664
107   24707
199   26069
301    2068
        index  num_x  num_y
type                       
101    101001    633  51240
101    101002    983  51240
101    101003  49431  51240
101    101004     19  51240
101    101005      3  51240
101    101006     14  51240
101    101007     17  51240
101    101008     33  51240
101    101009    107  51240
102    102001    215   2714
102    102

In [8]:
c['persentage'] = (c['num_x']/c['num_y'])*100
del c['num_y']
c.rename(columns={'num_x':'num'},inplace=True)
c

,index,num,persentage
type,,,
101,101001,633,1.235363
101,101002,983,1.918423
101,101003,49431,96.469555
101,101004,19,0.037080
101,101005,3,0.005855
101,101006,14,0.027322
101,101007,17,0.033177
101,101008,33,0.064403
101,101009,107,0.208821


## 1.4 网页107类型中的内部统计

In [9]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/wangye?charset=utf8')
sql = pd.read_sql('all_gzdata', engine, chunksize = 10000)

#统计107类别的情况
def count107(i): #自定义统计函数
  j = i[['fullURL']][i['fullURLId'].str.contains('107')].copy() #找出类别包含107的网址
  j['type'] = None #添加空列
  j['type'][j['fullURL'].str.contains('info/.+?/')] = u'知识首页' #info以/结尾
  j['type'][j['fullURL'].str.contains('info/.+?/.+?')] = u'知识列表页'
  j['type'][j['fullURL'].str.contains('/\d+?_*\d+?\.html')] = u'知识内容页'
  return j['type'].value_counts()

counts2 = [count107(i) for i in sql] #逐块统计
counts2 = pd.concat(counts2).groupby(level=0).sum() #合并统计结果

counts2.to_excel('./tmp/1_4type107.xlsx')
print(counts2)

D:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 478")
  result = self._query(query)


知识内容页    22566
知识列表页     1282
知识首页       859
Name: type, dtype: int64


# 2.点击次数分析
## 2.1 查看IP的出现次数

In [10]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/wangye?charset=utf8')
sql = pd.read_sql('all_gzdata', engine, chunksize = 10000)

#统计点击次数
c = [i['realIP'].value_counts() for i in sql] #分块统计各个IP的出现次数
count3 = pd.concat(c).groupby(level = 0).sum() #合并统计结果，level=0表示按index分组
count_df = pd.DataFrame(count3) #Series转为DataFrame
count3=count_df
count3[1]=1 # 添加1列全为1
print(count3.head())


        realIP  1
226318       2  1
270457       1  1
503676       1  1
611448       2  1
857460       1  1


## 2.2 点击次数与用户数量关系

In [11]:
realIP_sum=count3['realIP'].sum()
print(realIP_sum)
                                 
count3= count3.groupby('realIP').sum()##统计各个“不同点击次数”分别出现的次数# 也可以使用counts1_['realIP'].value_counts()功能

print(count3.head())

                                 
count3.columns=[u'用户数']
count3.index.name = u'点击次数'

print(count3.head())

107775
            1
realIP       
1       23861
2        6941
3        2568
4        1388
5         804
        用户数
点击次数       
1     23861
2      6941
3      2568
4      1388
5       804


### 2.2.1 用户百分比，点击记录百分比

In [12]:
count3[u'用户百分比'] = count3[u'用户数']/count3[u'用户数'].sum()*100
count3[u'点击记录百分比'] = count3[u'用户数']*count3.index/realIP_sum*100
count3.sort_index(inplace = True)
c=count3.iloc[:7,]
c=c.T
c

点击次数,1,2,3,4,5,6,7
用户数,23861.000000,6941.000000,2568.000000,1388.000000,804.000000,507.000000,323.000000
用户百分比,62.722780,18.245623,6.750434,3.648599,2.113454,1.332738,0.849062
点击记录百分比,22.139643,12.880538,7.148225,5.151473,3.729993,2.822547,2.097889


### 2.2.2 浏览七次以上数据合并

In [13]:
c.insert(0,u'总计',[count3[u'用户数'].sum(),100,100])
c[u'7次以上'] = c.iloc[:,0]- c.iloc[:,1:].sum(1)

c.to_excel('./tmp/2_2_2clickTimes.xlsx')
c

点击次数,总计,1,2,3,4,5,6,7,7次以上
用户数,38042,23861.000000,6941.000000,2568.000000,1388.000000,804.000000,507.000000,323.000000,1650.000000
用户百分比,100,62.722780,18.245623,6.750434,3.648599,2.113454,1.332738,0.849062,4.337311
点击记录百分比,100,22.139643,12.880538,7.148225,5.151473,3.729993,2.822547,2.097889,44.029691


In [14]:
# 转置表格，并将所有输出保留两位小数
d = c.T
format = lambda x: '%.2f' % x  # 也可以使用d.round(4)
d = d.applymap(format)
d.T

点击次数,总计,1,2,3,4,5,6,7,7次以上
用户数,38042.00,23861.00,6941.00,2568.00,1388.00,804.00,507.00,323.00,1650.00
用户百分比,100.00,62.72,18.25,6.75,3.65,2.11,1.33,0.85,4.34
点击记录百分比,100.00,22.14,12.88,7.15,5.15,3.73,2.82,2.10,44.03


### 2.2.3 浏览七次以上用户分析表

In [15]:
import pandas as pd
import numpy as np
# 分析浏览次数7次以上的数据
times = count3.index[7:]# [8,    9,   10,   11,   12,   13,   14, ...]
bins = [7,100,1000,50000]
cats = pd.cut(times,bins,right=True,labels=['8~100','101~1000','1000以上'])
#[8~100, 8~100, 8~100, 8~100, 8~100, ..., 101~1000, 101~1000, 101~1000, 1000以上, 1000以上]
e = cats.value_counts()
e = pd.DataFrame(e, columns =[u'用户数'])
e.index.name = u'点击次数'

e[u'用户数'] = np.nan
e.ix[u'8~100',u'用户数'] = count3.loc[8:100,:][u'用户数'].sum()
e.ix['101~1000',u'用户数'] = count3.loc[101:1000,:][u'用户数'].sum()
e.ix['1000以上',u'用户数'] = count3.loc[1001:,:][u'用户数'].sum()
e.sort_values(by=u'用户数',ascending=False,inplace = True)
e.reset_index(inplace=True)
e

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


,点击次数,用户数
0,8~100,1601.0
1,101~1000,47.0
2,1000以上,2.0


## 2.3 浏览一次的用户行为分析
### 2.3.1 点击次数为一的数据

In [16]:
# 获取浏览一次的所有数据
f = count_df[count_df['realIP']==1]
del f[1]
f.columns = [u'点击次数']
f.index.name = 'realIP'
f.head()

,点击次数
realIP,
270457,1
503676,1
857460,1
993851,1
1808142,1


### 2.3.2 合并数据

In [17]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/wangye?charset=utf8')
sql = pd.read_sql('all_gzdata', engine, chunksize = 10000)
# g = [pd.merge(f,i[['fullURLId','fullURL','realIP']],right_on = 'realIP',left_index=True,how ='left') for i in sql]
g = [i[['fullURLId','fullURL','realIP']] for i in sql]
g = pd.concat(g)
h = pd.merge(f,g,right_on = 'realIP',left_index=True,how ='left')
h.head()

D:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 478")
  result = self._query(query)


,点击次数,fullURLId,fullURL,realIP
8355,1,101003,http://www.lawtime.cn/ask/question_2077376.html,270457
6285,1,1999001,http://www.lawtime.cn/ask/exp/13445.html,503676
7924,1,107001,http://www.lawtime.cn/info/hetong/htfanben/qit...,857460
4204,1,1999001,http://www.lawtime.cn/ask/exp/8191.html,993851
89,1,101003,http://www.lawtime.cn/ask/question_379149.html,1808142


### 2.3.4 浏览一次的用户的网页类型ID分析

In [18]:
# 浏览一次的用户的网页类型ID分析
i = h['fullURLId'].value_counts()
i = pd.DataFrame(i)
print(i)
i.rename(columns={'fullURLId':u'个数'},inplace=True)
i.index.name = u'网页类型ID'
i[u'百分比'] = i[u'个数']/i[u'个数'].sum()*100
 
#保存的表名命名格式为“1_2_k此表功能名称”，此表表示生成的第2张表格，功能为typeID：浏览一次的用户的网页类型ID分析
i.to_excel('./tmp/2_3_4typeID.xlsx')
i

         fullURLId
101003       17838
107001        3930
1999001       1952
301001          99
102001          12
106001          10
101002           9
101001           5
102002           3
101009           2
103003           1


,个数,百分比
网页类型ID,,
101003,17838,74.757973
107001,3930,16.470391
1999001,1952,8.180713
301001,99,0.414903
102001,12,0.050291
106001,10,0.041909
101002,9,0.037718
101001,5,0.020955
102002,3,0.012573


In [19]:
j=i[i[u'个数']>100]

j.loc[u'其他',u'个数'] = i[i[u'个数']<=100][u'个数'].sum()
j.loc[u'其他',u'百分比'] = i[i[u'个数']<100][u'百分比'].sum()
j# 浏览一次的用户中浏览的网页类型ID

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,个数,百分比
网页类型ID,,
101003,17838.0,74.757973
107001,3930.0,16.470391
1999001,1952.0,8.180713
其他,141.0,0.590922


### 2.3.5 点击1次用户浏览网页统计(点击数大于100次的）

In [20]:
#　点击1次用户浏览网页统计(点击数大于100次的)
k = pd.DataFrame(h['fullURL'].value_counts())
k.index.name = u'网址'
k.columns = [u'点击数']
m = k[k[u'点击数'] > 100]
m.loc[u'其他',u'点击数'] = k[k[u'点击数']<=100][u'点击数'].sum()
m[u'百分比'] = m[u'点击数']/k[u'点击数'].sum()
#保存的表名命名格式为“1_2_k此表功能名称”，此表表示生成的第3张表格，功能为lookMorethan100：点击1次用户浏览网页统计(点击数大于100次的)
m.to_excel('./tmp/2_3_5lookMorethan100.xlsx')
m

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,点击数,百分比
网址,,
http://www.lawtime.cn/info/shuifa/slb/2012111978933.html,224.0,0.009388
http://www.lawtime.cn/ask/exp/13655.html,154.0,0.006454
http://www.lawtime.cn/ask/question_925675.html,141.0,0.005909
http://www.lawtime.cn/ask/exp/8495.html,109.0,0.004568
http://www.lawtime.cn/info/shuifa/slb/2012111978933_2.html,106.0,0.004442
其他,23127.0,0.969239
